# Sistema de Recomendação

Este notebook apresenta algumas implementações de sistema de recomendação utilizando
o dataset Movielens.
Este notebook contém:
- Modelo de Sistema de Recomendação utilizando atributos latentes e produto interno
- Modelo utilizando atributos latentes concatenados e uma rede neural
- Calcula a predição para todos os filmes e usuários que não fizeram suas avaliações
- Visualiza a matriz de avaliações por usuários x filmes com todas as predições
- Análise do significado dos embeddings

Obs: Este notebook foi inspirado em exemplo divulgado pelo curso online disponível em [fast.ai](http://fast.ai)

## Importação da bibliotecas

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os, sys
import pandas as pd

In [2]:
import torch
from torch import nn
from torch.autograd import Variable

In [3]:
np.set_printoptions(formatter={'float': '{: 0.1f}'.format})

# from course libs
sys.path.append('./lib')
from pytorch_utils import DeepNetTrainer

## Movielens dataset

- [Movielens - Readme](http://files.grouplens.org/datasets/movielens/ml-latest-small-README.html)
- Movielens - Small data set - 100.000 avaliações:[ml-latest-small.zip](http://files.grouplens.org/datasets/movielens/ml-latest-small.zip)

F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015).


### Leitura dos Dados - Movielens dataset

Caso a célula a seguir falhar, é necessário executar o notebook:
- [Movielens Dataset](MovieLens_dataset.ipynb) que é responsável por ler o dataset original e
prepará-lo para este notebook

Este dataset consiste de:
- 100.004 avaliações de filmes (amostras), feitas por
- 671 usuários, sobre
- 9066 filmes

As amostras estão no array `ratings` organizados da seguinte forma:
- cada linha é uma amostra
- coluna 0 é o iD do usuário
- coluna 1 é o iD do filme
- coluna 2 é a avaliação do usuários entre 0.0 e 5.0

In [4]:
data = np.load('../data/movielens_norm.npz')
ratings = data['ratings']
movie_names = data['movie_names']
n_samples = ratings.shape[0]
print('n_samples:',n_samples)
print('ratings:\n', ratings[:5])

n_samples: 100004
ratings:
 [[ 0.0  30.0  2.5]
 [ 0.0  833.0  3.0]
 [ 0.0  859.0  3.0]
 [ 0.0  906.0  2.0]
 [ 0.0  931.0  4.0]]


In [5]:
for i in (ratings[:5,1]).astype(np.int):
    print(movie_names[i])

Dangerous Minds (1995)
Dumbo (1941)
Sleepers (1996)
Escape from New York (1981)
Cinema Paradiso (Nuovo cinema Paradiso) (1989)


In [6]:
h_userId = np.bincount(ratings[:,0].astype(np.int))
n_users = h_userId.size
h_movieId = np.bincount(ratings[:,1].astype(np.int))
n_movies = h_movieId.size
print('n_users:',n_users)
print('n_movies:',n_movies)

n_users: 671
n_movies: 9066


## Divisão dos dados em treinamento e validação

In [7]:
np.random.seed = 42

Faz a divisão com 80% das amostras para treinamento e 20% para validação:

In [8]:
msk = np.random.rand(n_samples) < 0.8
train = ratings[msk]
valid = ratings[~msk]
print('train:',train.shape[0],' amostras:\n',train[:5])
print('valid:',valid.shape[0],' amostras:\n',valid[:5])

train: 80005  amostras:
 [[ 0.0  30.0  2.5]
 [ 0.0  833.0  3.0]
 [ 0.0  859.0  3.0]
 [ 0.0  906.0  2.0]
 [ 0.0  1017.0  2.0]]
valid: 19999  amostras:
 [[ 0.0  931.0  4.0]
 [ 0.0  1140.0  1.0]
 [ 0.0  1665.0  4.0]
 [ 1.0  37.0  5.0]
 [ 1.0  45.0  4.0]]


`userId` e `movieId` precisam ser inteiros pois são entradas do *Embedding*:

In [9]:
train_userId =  torch.LongTensor(train[:,0].astype(np.int))
train_movieId = torch.LongTensor(train[:,1].astype(np.int))
train_ratings = torch.FloatTensor(train[:,2:3]) # importante que fique bidimensional
valid_userId =  torch.LongTensor(valid[:,0].astype(np.int))
valid_movieId = torch.LongTensor(valid[:,1].astype(np.int))
valid_ratings = torch.FloatTensor(valid[:,2:3]) # importante que fique bidimensional

## Definição da classe MLDataset

In [10]:
from torch.utils.data import Dataset
class MLDataset(Dataset):
    
    def __init__(self, data_user, data_movie, target):
        assert data_user.size(0) == target.size(0) and \
               data_movie.size(0) == target.size(0)
        
        self.data = torch.transpose(torch.stack( (data_user, data_movie) ),0,1)
        self.target = target
    
    def __len__(self):
        return self.target.size(0)
    
    def __getitem__(self, i):
        return (self.data[i], self.target[i])

### Criação dos objetos datasets e dataloaders

In [11]:
datasets = {
    'train': MLDataset(train_userId, train_movieId, train_ratings),
    'val'  : MLDataset(valid_userId, valid_movieId, valid_ratings)
    }

dataloaders = {
    'train': torch.utils.data.DataLoader(datasets['train'], batch_size=64, shuffle=True, num_workers=4),
    'val'  : torch.utils.data.DataLoader(datasets['val'], batch_size=64, shuffle=True, num_workers=4)
    }

dataset_sizes = {
    'train': len(datasets['train']),
    'val'  : len(datasets['val'])
    }

print(dataset_sizes)

{'train': 80005, 'val': 19999}


### Testando os datasets

In [25]:
x, y = datasets['train'][3:4]
print(x)
print(y)


   0  906
[torch.LongTensor of size 1x2]


 2
[torch.FloatTensor of size 1x1]



### Testando os dataloaders

In [26]:
batch_size = 64
print( len(dataloaders['val']) * batch_size ) # verificando mini-batches
for k,data in enumerate(dataloaders['val']):
    print(k, len(data[0]), len(data[1]))
    if k > 1: break

20032
0 64 64
1 64 64
2 64 64


## Primeira solução - usando produto interno: Dot

<img src='../figures/Recomendacao_dot.png', width=600ptx></img>

O primeiro modelo é o produto interno entre os atributos latentes dos usuários e dos filmes.
Este produto interno é implementado pela operação `dot` do Keras. Como o modelo agora não é
sequencial, há necessidade de utilizarmos o modelo API:

In [14]:
class DotNet(nn.Module):
    def __init__(self, n_users, n_movies, n_attributes):
        """
        No construtor, criamos as duas embeddings, uma para os usuários e 
        outra para os filmes.
        """
        super(DotNet, self).__init__()
        self.user_emb  = nn.Embedding(n_users, n_attributes)
        self.movie_emb = nn.Embedding(n_movies, n_attributes)

    def forward(self, x):
        """
        x: (user_id, movie_id)
        """
        user_id  = x[:,0]
        movie_id = x[:,1]
        user_attr  = self.user_emb(user_id)
        movie_attr = self.movie_emb(movie_id)
        y_pred = (user_attr * movie_attr).sum(1)
        return y_pred


### Instanciando a rede

In [15]:
n_factors = 50
model_dot = DotNet(n_users, n_movies, n_factors)
model_dot

DotNet (
  (user_emb): Embedding(671, 50)
  (movie_emb): Embedding(9066, 50)
)

### Testando o predict e o loss da rede com poucos dados e sem treinar

In [16]:
criterion = nn.MSELoss(size_average=True)
data_um = torch.transpose(torch.stack( (train_userId[0:3],train_movieId[0:3]) ),1,0)
print(data_um)
y_pred = model_dot(Variable(data_um))
print(y_pred)
loss = criterion(y_pred, Variable(train_ratings[0:3]))
print(loss)


   0   30
   0  833
   0  859
[torch.LongTensor of size 3x2]

Variable containing:
-8.9329
-3.2705
-5.3418
[torch.FloatTensor of size 3]

Variable containing:
 79.8723
[torch.FloatTensor of size 1]



### Testando o predict e loss com dataset

In [17]:
x,y = datasets['train'][3:4]
print(x)
print(y)
y_pred = model_dot(Variable(x))
print(y_pred)
loss = criterion(y_pred, Variable(y))
print(loss)


   0  906
[torch.LongTensor of size 1x2]


 2
[torch.FloatTensor of size 1x1]

Variable containing:
-2.7927
[torch.FloatTensor of size 1]

Variable containing:
 22.9697
[torch.FloatTensor of size 1]



### Testando o predict e o dataset

### Treinando a rede

Verificando se um pequeno conjunto de dados consegue treinar a rede

In [57]:
batch_size = 6400
criterion = nn.MSELoss()
#optimizer = torch.optim.SGD(model_dot.parameters(), lr=1e-1)
optimizer = torch.optim.Adam(model_dot.parameters())
for t in range(10):
    # Forward pass: Compute predicted y by passing x to the model
    data_um = torch.transpose(torch.stack( (train_userId[0:batch_size],
                                            train_movieId[0:batch_size]) ),0,1)

    y_pred = model_dot((Variable(data_um)))

    # Compute and print loss
    loss = criterion(y_pred, Variable(train_ratings[:batch_size]))
    print(t, loss.data[0])

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 26.89550018310547
1 26.700855255126953
2 26.50751304626465
3 26.315481185913086
4 26.124956130981445
5 25.935815811157227
6 25.748126983642578
7 25.5617733001709
8 25.376920700073242
9 25.193567276000977


In [58]:
a = torch.pow(y_pred.data - train_ratings[:batch_size],2).mean()
a = torch.pow(y_pred.data ,2).mean()
a

14.412419885018407

In [59]:
trainer = DeepNetTrainer(file_basename='../../models/Recomendacao_dot', 
                         model=model_dot, 
                         criterion=criterion, 
                         optimizer=optimizer, 
                         #lr_scheduler=exp_lr_scheduler,
                         #metrics=metrics,
                         reset=False)

Model loaded from ../../models/Recomendacao_dot.model


In [ ]:
trainer.fit(n_epochs=200, 
            train_data = dataloaders['train'], 
            valid_data = dataloaders['val'])

Starting training for 200 epochs

 37:   0.0s   T: 32.59958   V: 39.09452 best
 38:  11.4s   T: 30.80627   V: 35.53824 best
 39:  17.7s   T: 25.27228   V: 32.82726 best
 40:  17.2s   T: 20.94904   V: 30.59887 best
 41:  17.3s   T: 17.50834   V: 28.64466 best
 42:  17.5s   T: 14.56231   V: 26.79902 best
 43:  18.7s   T: 11.96924   V: 24.84748 best
 44:  18.6s   T: 9.57837   V: 22.77557 best
 45:  18.9s   T: 7.39973   V: 20.58994 best
 46:  18.7s   T: 5.54455   V: 18.55400 best
 47:  18.8s   T: 4.09127   V: 16.76489 best
 48:  18.9s   T: 3.03149   V: 15.29595 best
 49:  18.7s   T: 2.27597   V: 14.11400 best
 50:  19.0s   T: 1.74406   V: 13.16956 best
 51:  18.9s   T: 1.36700   V: 12.38711 best
 52:  18.8s   T: 1.09069   V: 11.72898 best
 53:  19.1s   T: 0.89112   V: 11.20232 best
 54:  19.0s   T: 0.74250   V: 10.75294 best
 55:  19.0s   T: 0.63340   V: 10.39412 best
 56:  19.2s   T: 0.54887   V: 10.05649 best
 57:  19.6s   T: 0.48588   V: 9.81799 best
 58:  19.0s   T: 0.43647   V: 9.5938

### Avaliando a rede

In [ ]:
model = test_network(model_name, [valid_userId, valid_movieId], valid_ratings)

Os bons modelos [best benchmarks](http://www.librec.net/example.html) são próximos de 0.9, há necessidade de melhorias.

### Predição de usuário e filme

Para calcular qualquer predição de qualquer usuário e qualquer filme, usa-se o `predict` do modelo treinado:

In [ ]:
model.predict([np.array([3]), np.array([6])])

##  Usado Rede Neural

<img src='../figures/Recomendacao_NN.png', width=600ptx></img>

Uma solução usando rede neural é concatenar a saída dos embeddings e em seguida colocar uma camada densa
antes da última camada de um neurônio. No exemplo a seguir foi utilizado uma camada de 70 neurônios.

In [ ]:
user_in = Input(shape=(1,),dtype='int64', name='user_in')
u = Embedding(n_users, n_factors, input_length=1, 
              embeddings_regularizer=l2(1e-4))(user_in)

movie_in = Input(shape=(1,),dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factors, input_length=1, 
              embeddings_regularizer=l2(1e-4))(movie_in)

In [ ]:
x = keras.layers.concatenate([u, m])
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(70, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)
model_nn = Model([user_in, movie_in], x)
print(model_nn.summary())

### Treinando a rede

In [ ]:
model_name = '../../models/Recomendacao_nn'
fit_params = {
    'model_name': model_name,
    'loss':       'mse',
    'opt':        Adam(), 
    'batch_size': 64, 
    'nepochs':    10,
    'patience':   5,
    'ploss':      3.,
    'reset':      False,
}

train_network(model_nn, 
              [train_userId, train_movieId], train_ratings, 
              [valid_userId, valid_movieId], valid_ratings, **fit_params);

### Avaliando a rede

In [ ]:
model_nn = test_network(model_name, [valid_userId, valid_movieId], valid_ratings)

In [ ]:
model_nn.predict([np.array([3]), np.array([6])])

Com esta rede, a perda já é bem melhor, comparável com os melhores sistemas de recomendação.

## Matriz de recomendações, por usuário e por filme

O sistema de recomendação pode ser visualizado por uma matriz onde as linhas sejam os
IDs dos usuários e as colunas sejam os IDs dos títulos dos filmes. Colocamos como -1
os elementos em que não existem avaliações. Esta matriz é bastante esparsa, pois existem
normalmente poucas avaliações feitas.

In [ ]:
grid_ratings = -1. * np.ones((n_users,n_movies))
uId = (ratings[:,0]).astype(np.int)
mId = (ratings[:,1]).astype(np.int)
grid_ratings[uId,mId] = ratings[:,2] # Criação da matriz
print(grid_ratings.shape)

### Visualização da matriz de recomendações, original

In [ ]:
show_ratings = np.zeros((n_users,n_movies,3))
show_ratings[:,:,0] = np.where(grid_ratings == -1., 5, grid_ratings)
show_ratings[:,:,1] = np.where(grid_ratings == -1., 0., grid_ratings)
show_ratings[:,:,2] = np.where(grid_ratings == -1., 0., grid_ratings)


import matplotlib.pyplot as plt
plt.figure(figsize=(7,7))
plt.title('usuários (linhas) x filmes (colunas)')
plt.imshow(show_ratings[:150,:150,:])
plt.xlabel('filmes')
plt.ylabel('usuários')
plt.show()

## Predições para todos os usuários e filmes

In [ ]:
n2p_user,n2p_movie = np.nonzero(grid_ratings==-1)
recommend = model_nn.predict([n2p_user, n2p_movie])

### Montagem da matriz de recomendação "cheia"

In [ ]:
gg = grid_ratings.copy()
gg[n2p_user,n2p_movie] = recommend[:,0]

plt.figure(figsize=(8,8))
plt.title('usuários (linhas) x filmes (colunas)')
plt.xlabel('usuarios')
plt.ylabel('filmes')
plt.imshow(gg[:150,:150],cmap='gray')
plt.show()

### Visualização de uma parte da matriz

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(show_ratings[:150,:150,:])

### Visualização da parte de usuários mais ativos e filmes mais populares

In [ ]:
io_popular_movies = np.argsort(h_movieId)[::-1]
io_top_users = np.argsort(h_userId)[::-1].astype(np.int)
gg_ord = gg[np.ix_(io_top_users,io_popular_movies)]
plt.figure(figsize=(7,7))
plt.title('usuários (linhas) x filmes (colunas)')
plt.imshow(gg_ord[:150,:150],cmap='gray')
plt.show()

### Visualização dos usuários menos ativos e filmes menos avaliados

In [ ]:
plt.figure(figsize=(7,7))
plt.title('usuários (linhas) x filmes (colunas)')
plt.imshow(gg_ord[-150:,-150:],cmap='gray')
plt.show()

## Analise dos embeddings dos filmes

A análise a seguir será feita apenas com os 2000 filmes mais populares:

In [ ]:
topMovies = io_popular_movies[:2000]

### Extração dos atributos latentes (embeddings) dos 2000 fimes mais populares

Para obter os atributos latentes dos 2000 filmes mais populares, primeiro criamos uma
nova rede, denominada `get_movie_emb`, a partir da rede `Model`, com a entrada apenas o ID dos filmes e
a saída `m`, após o embedding. Aplicamos a predição desta rede nos `topMovies`:

In [ ]:
get_movie_emb = Model(movie_in, m)
m_emb = get_movie_emb.predict([topMovies])
movie_emb = np.squeeze(m_emb) # elimina dimensões 1
print(m_emb.shape)
movie_emb.shape

Como o embedding de cada filme tem dimensão 50, é muito difícil conseguir analisá-lo desta forma.
Uma forma muito usual é reduzir esta dimensionalidade utilizando uma técnica denominada PCA -
Principal Component Analysis: [PCA](https://plot.ly/ipython-notebooks/principal-component-analysis/).
Iremos reduzir a dimensão dos embeddings de 50 para 3. 

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
movie_pca = pca.fit(movie_emb.T).components_

#### Filmes com alto valor na primeira dimensão do PCA

In [ ]:
fac0 = movie_pca[0]
isort = np.argsort(fac0)[::-1]

for ii in isort[:15]:
    print(fac0[ii],movie_names[ii])

#### Filmes com baixo valor na primeira dimensão do PCA

In [ ]:
for ii in isort[-15:]:
    print(fac0[ii],movie_names[ii])

### Análise da segunda dimensão do PCA

In [ ]:
fac1 = movie_pca[1]
isort = np.argsort(fac1)[::-1]

#### Mais bem avaliados na segunda dimensão

In [ ]:
for ii in isort[:15]:
    print(fac1[ii],movie_names[ii])

#### Piores avaliados na segunda dimensão

In [ ]:
for ii in isort[-15:]:
    print(fac1[ii],movie_names[ii])

### Análise da terceira dimensão do PCA

In [ ]:
fac2 = movie_pca[2]
isort = np.argsort(fac2)[::-1]

#### Mais bem avaliados na terceira dimensão do PCA

In [ ]:
for ii in isort[:15]:
    print(fac2[ii],movie_names[ii])

#### Piores avaliados na terceira dimensão do PCA

In [ ]:
for ii in isort[-15:]:
    print(fac2[ii],movie_names[ii])

### Visualizando duas dimensões do PCA

In [ ]:
start=50; end=100
X = fac0[start:end]
Y = fac2[start:end]
plt.figure(figsize=(15,15))
plt.scatter(X, Y)
for i, x, y in zip(topMovies[start:end], X, Y):
    plt.text(x,y,movie_names[i], color=np.random.rand(3)*0.7, fontsize=14)
plt.show()

## Exercícios

1. Inclua um novo usuário, sem nenhuma avaliação. Treine a rede e verifique
   se após a rede treinada, se haverá alguma avaliação.
2. Com o novo usuário, faça uma única avaliação e verifique quais os 10 filmes
   mais recomendados para ele.

## Aprendizados com este notebook